In [1]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-shus55f2/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-shus55f2/python-terrier
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104733 sha256=5a5567f04616d5b9a1eae73cf0b546de416a8a8b5b87c68a19b9092e7a8f5525
  Stored in directory: /tmp/pip-ephem-wheel-cache-drm6fn9u/wheels/92/00/3b/91460ee205c531cfa11bb4da2b9804ff1a4d57d77d175e7496
Successfully built python-terrier


  Attempting uninstall: python-terrier
    Found existing installation: python-terrier 0.8.1
    Uninstalling python-terrier-0.8.1:
      Successfully uninstalled python-terrier-0.8.1


In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:
import pyterrier as pt
from pyterrier_monoQA import MonoQA
monoqa = MonoQA()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

In [ ]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "text") >> monoqa

In [ ]:
mono_pipeline.transform(dataset.get_topics().head())